# DATA MANAGEMENT FUNDAMENTALS ASSIGNMENT

In [25]:
#importing libraries
import pandas as pd
import sqlite3

In [26]:
#reading the data
air_data = pd.read_csv("Air_Quality_Continuous.csv")

In [27]:
#data summary
air_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1603492 entries, 0 to 1603491
Data columns (total 19 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Date_Time    1603492 non-null  object 
 1   Site_ID      1603483 non-null  float64
 2   NOx          1467692 non-null  float64
 3   NO2          1464857 non-null  float64
 4   NO           1473978 non-null  float64
 5   PM10         364710 non-null   float64
 6   O3           269748 non-null   float64
 7   Temperature  107573 non-null   float64
 8   ObjectId     0 non-null        float64
 9   ObjectId2    1603492 non-null  int64  
 10  NVPM10       88165 non-null    float64
 11  VPM10        88158 non-null    float64
 12  NVPM2_5      71978 non-null    float64
 13  PM2_5        143083 non-null   float64
 14  VPM2_5       71978 non-null    float64
 15  CO           195930 non-null   float64
 16  RH           33450 non-null    float64
 17  Pressure     0 non-null        float64
 18  SO

From the above summary, we can see that there are `19 columns` in total. Three data types are  used: `int64`, `float64`, and `object`. This makes it easier for us to define the data types of the columns when creating a `database` using `MySQL Workbench`

In [28]:
#checking the first 10 rows
air_data.head(10)

,Date_Time,Site_ID,NOx,NO2,NO,PM10,O3,Temperature,ObjectId,ObjectId2,NVPM10,VPM10,NVPM2_5,PM2_5,VPM2_5,CO,RH,Pressure,SO2
0,2019/04/29 23:00:00+00,501.0,122.25,49.25,47.75,30.400000,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019/04/30 03:00:00+00,501.0,42.25,35.25,4.50,20.275000,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019/05/01 06:00:00+00,501.0,82.00,48.50,21.75,28.125000,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019/05/01 18:00:00+00,501.0,357.25,117.75,156.50,27.700001,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019/05/02 14:00:00+00,501.0,194.50,73.25,79.25,23.775000,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019/05/02 17:00:00+00,501.0,236.25,89.50,95.75,22.924999,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2019/05/03 02:00:00+00,501.0,63.25,36.50,17.25,16.549999,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2019/05/03 03:00:00+00,501.0,71.00,38.00,21.25,16.075001,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2019/05/03 07:00:00+00,501.0,376.00,92.75,184.75,18.375000,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019/05/03 14:00:00+00,501.0,274.75,89.00,121.00,18.625000,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#function to return the min and max date in our data
def min_max_time(df,column):
    min_value = df[column].min()
    max_value = df[column].max()
    
    return (min_value, max_value)


min_max_time(air_data, "Date_Time")

('1993/01/01 00:00:00+00', '2611/02/05 08:03:20+00')

### CROPPING AND CLEANSING THE DATA
Once the data is collected, it is essential to clean and preprocess it before using it for analysis. In our case we will crop the dataset to hold only the data from 1st January 2015 on. This is because the data from before this date is not relevant to our analysis/objective of the assignment. After cropping, we will cleanse the cropped dataset to ensure that all dates fall between 1st January 2015 and 22nd October 2023.

Since we are cropping the data based on the date, we will have to convert the  date column to `datetime` format. We will use the `pandas` library to achieve this. From the above function `min_max_time` we can notice that the maximum date object is not a valid date. To avoid errors while converting the column to datetime datatype, we pass the argument `coerce`  to `pd.to_datetime()` function. This will convert the invalid date to `NaT`

In [30]:
#changing Date_time from object to datetime dtype
air_data["Date_Time"] = pd.to_datetime(air_data["Date_Time"], errors= "coerce")

#getting rid of utc time format
air_data["Date_Time"] = air_data["Date_Time"].dt.tz_localize(None)
#cropping the data
uncleaned_cropped_data  = air_data[air_data["Date_Time"] >= "2015-01-01"]

Our dataset has been cropped and by looking at the above information we now have `520833` rows of data unlike the original dataset which had `1603492`

Next is to clean the data before we can populate it to our database. I will also check the min and max values of Date_Time  column to see if there are any anomalies. 

In [31]:
min_max_time(uncleaned_cropped_data, "Date_Time")

(Timestamp('2015-01-01 00:00:00'), Timestamp('2023-10-22 14:00:00'))

In [32]:
#checking if our date_time column has NAT values
nat_count = uncleaned_cropped_data["Date_Time"].isna().sum()
print("Number of NaT values:",nat_count)

Number of NaT values: 0


Our cropped data now falls within the specified time in our assignment; `1st January 2015 to 22nd October 2022`, it also has no invalid dates(`NAT). We will then sort the data by date in descending order since we want the most recent dates to be displayed first. We will also reset the  index to ensure that the index is continuous and starts from 0.

In [33]:
#sorting data by date
uncleaned_cropped_data = uncleaned_cropped_data.sort_values(by= "Date_Time",  ascending = False)
#resetting index
uncleaned_cropped_data.reset_index(drop= True, inplace= True)

# cropped_data.dropna(inplace= True)

We will now save our cropped data into a csv file

In [34]:
uncleaned_cropped_data.index.name = 'ObservationID'

uncleaned_cropped_data.to_csv("cropped.csv", index= True)

In [35]:
site_count = uncleaned_cropped_data["Site_ID"].isna().sum()
print("Number of NaT values:",site_count)

Number of NaT values: 9


In [36]:
uncleaned_cropped_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520833 entries, 0 to 520832
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Date_Time    520833 non-null  datetime64[ns]
 1   Site_ID      520824 non-null  float64       
 2   NOx          493537 non-null  float64       
 3   NO2          493567 non-null  float64       
 4   NO           493532 non-null  float64       
 5   PM10         131522 non-null  float64       
 6   O3           69647 non-null   float64       
 7   Temperature  107573 non-null  float64       
 8   ObjectId     0 non-null       float64       
 9   ObjectId2    520833 non-null  int64         
 10  NVPM10       24710 non-null   float64       
 11  VPM10        24710 non-null   float64       
 12  NVPM2_5      23580 non-null   float64       
 13  PM2_5        94685 non-null   float64       
 14  VPM2_5       23580 non-null   float64       
 15  CO           0 non-null       floa

In [37]:
uncleaned_cropped_data["Site_ID"].unique()

array([672., 463., 270., 501., 203., 215.,  nan, 452., 500., 375., 206.])

After cropping our data we can see that we have `11` stations remaining from the initial `19` that we had. This can indicate that only 10 weather sites were working since ` 1st January 2015 to 22nd October 2023`. We can see we also have a `nan` value in the `Site_ID` column, this is impossible since a weather site cannot be `nan`. This is likely due to a data entry error. We will interact with our data using `SQLite3` to see if there is a pattern for the missing value.

SQLite3 will allow us to write SQL queries in our notebook and manipulate our cropped data as much as we want.

In [38]:
#creating an in-memory database
connection = sqlite3.connect(":memory:")
#connecting with our DataFrame
uncleaned_cropped_data.to_sql("pollution", connection, index= True, if_exists= "replace")

520833

We now have our data in my computer's memory as a database. We will statrt writing `SQl` queries to interact with our data. We will first doublecheck the distinct(unique) `Site_ID` values. I will define a function first that takes in the query and the database. This function will return the results of the query.

In [39]:
#query function
def sql_query(query, conn):
    
    return pd.read_sql(query, conn)

In [40]:
#first query
query_1  = "SELECT DISTINCT Site_ID FROM pollution"
#call query function
sql_query(query_1, connection)

,Site_ID
0,672.0
1,463.0
2,270.0
3,501.0
4,203.0
5,215.0
6,NaN
7,452.0
8,500.0
9,375.0


Next we will write a query that returns the dates of the `nan` values in our data

In [41]:
#second_query
query_2 = "SELECT Date_time FROM pollution WHERE Site_ID IS NULL"
#call query function
sql_query(query_2, connection)

,Date_Time
0,2023-05-25 15:00:00
1,2023-05-25 15:00:00
2,2023-05-25 15:00:00
3,2023-05-25 15:00:00
4,2023-05-25 15:00:00
5,2023-05-25 15:00:00
6,2023-05-25 15:00:00
7,2023-05-25 15:00:00
8,2023-05-25 15:00:00


Voila! We know know the date and time of the 9 records that are missing the `Site_ID` value. Next we will write a query that returns all the records where the date is similar to that of the records which has missing Id's, in this case `2023-05-25 15:00:00`

In [42]:
#third_querry
query_3 = "SELECT * FROM pollution WHERE Date_time = '2023-05-25 15:00:00'"
#call query function
sql_query(query_3, connection)

,ObservationID,Date_Time,Site_ID,NOx,NO2,NO,PM10,O3,Temperature,ObjectId,ObjectId2,NVPM10,VPM10,NVPM2_5,PM2_5,VPM2_5,CO,RH,Pressure,SO2
0,19557,2023-05-25 15:00:00,215.0,36.969687,24.933688,7.846401,None,None,None,None,1565275,None,None,None,None,None,None,NaN,None,None
1,19558,2023-05-25 15:00:00,463.0,36.141469,22.078750,9.166143,None,None,None,None,1565285,None,None,None,None,None,None,NaN,None,None
2,19559,2023-05-25 15:00:00,463.0,36.141469,22.078750,9.166143,None,None,None,None,1565274,None,None,None,None,None,None,NaN,None,None
3,19560,2023-05-25 15:00:00,270.0,23.594406,15.004094,5.603187,None,None,None,None,1565276,None,None,None,None,None,None,NaN,None,None
4,19561,2023-05-25 15:00:00,203.0,12.662344,10.563375,1.366158,None,None,None,None,1565277,None,None,None,None,None,None,NaN,None,None
5,19562,2023-05-25 15:00:00,672.0,59.646792,32.067802,17.971348,None,None,None,None,1565278,None,None,None,None,None,None,NaN,None,None
6,19563,2023-05-25 15:00:00,463.0,36.141469,22.078750,9.166143,None,None,None,None,1565279,None,None,None,None,None,None,NaN,None,None
7,19564,2023-05-25 15:00:00,215.0,36.969687,24.933688,7.846401,None,None,None,None,1565280,None,None,None,None,None,None,27.0,None,None
8,19565,2023-05-25 15:00:00,270.0,23.594406,15.004094,5.603187,None,None,None,None,1565281,None,None,None,None,None,None,NaN,None,None
9,19566,2023-05-25 15:00:00,203.0,12.662344,10.563375,1.366158,None,None,None,None,1565282,None,None,None,None,None,None,NaN,None,None


From the above output, if we inspect carefully we can see that some rows have the same values with the records that had a missing Site_ID. For instance, record number `28` of observationID `19585` with a missing `Site_ID`, has the same values as record number `18` of observationID `19575` with `Site_ID 463`. The only thing that makes this two records different is `ObjectId2`. This helps us know that the missing `Site_ID` in record number `28` is likely `463`. We can use this information to fill in the missing `Site_ID` in the original dataframe. We will the same technique to fill in the missing `Site_ID` in the rest of the records.

Below we will write a function that takes in a SQL query and a dataframe as input, and returns the dataframe with the missing `Site_ID` filled in.

In [43]:
#query_4
query_4 = """UPDATE pollution AS p1
    SET Site_id = (
    SELECT p2.Site_id
    FROM pollution AS p2
    WHERE p1.NOx = p2.NOx
    AND p1.NO2 = p2.NO2
    AND p2.site_id IS NOT NULL
    )
    WHERE p1.site_id IS NULL;
    """
#updating our database
with connection:
    connection.execute(query_4)

After updating our database we will now save our updated database to a csv file so we can use it to populate our database in `php Myadmin`. 

In [44]:
#saving updated DB to CSV
query_5 = "SELECT * FROM pollution"
updated_data = pd.read_sql(query_5, connection)
#saving to csv
updated_data.to_csv("cropped_and_cleaned.csv", index=False)

Confirming if the data has been updated and we have a new version of the data that has no `nan` values on the `Site_ID` coliumn

In [45]:
updated_data["Site_ID"].unique()

array([672., 463., 270., 501., 203., 215., 452., 500., 375., 206.])

`PERFECT!!!` That marks the end of my cleaning process and I can now proceed to populating the database with the data.

In [46]:
updated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520833 entries, 0 to 520832
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ObservationID  520833 non-null  int64  
 1   Date_Time      520833 non-null  object 
 2   Site_ID        520833 non-null  float64
 3   NOx            493537 non-null  float64
 4   NO2            493567 non-null  float64
 5   NO             493532 non-null  float64
 6   PM10           131522 non-null  float64
 7   O3             69647 non-null   float64
 8   Temperature    107573 non-null  float64
 9   ObjectId       0 non-null       object 
 10  ObjectId2      520833 non-null  int64  
 11  NVPM10         24710 non-null   float64
 12  VPM10          24710 non-null   float64
 13  NVPM2_5        23580 non-null   float64
 14  PM2_5          94685 non-null   float64
 15  VPM2_5         23580 non-null   float64
 16  CO             0 non-null       object 
 17  RH             33450 non-null

In [48]:
#terminating the sqlite3 connection
connection.close()

I will use another notebook for importing data to my database. `import.ipynb` is the name of the notebook